Load libraries

In [1]:
require(ggplot2)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(car)
require(boot)
require(dplyr)
require(sjstats)
require(broom)

Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.4.4"Loading required package: reshape2
Warning message:
"package 'reshape2' was built under R version 3.4.4"Loading required package: lme4
Warning message:
"package 'lme4' was built under R version 3.4.4"Loading required package: Matrix
Loading required package: compiler
Loading required package: parallel
Loading required package: car
Loading required package: boot

Attaching package: 'boot'

The following object is masked from 'package:car':

    logit

Loading required package: dplyr

Attaching package: 'dplyr'

The following object is masked from 'package:car':

    recode

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: sjstats
Warning message:
"package 'sjstats' was built under R version 3.4.4"Warning message in checkMatrixPackageVersion():

# Load data and set factors

In [2]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# sid is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$cvs_graph <- factor(mydata$cvs_graph)
mydata$cvs_table <- factor(mydata$cvs_table)

In [3]:
head(mydata)
colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,pre_with_ident,main_with_ident,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars
10127163,L,Concentration,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,L,Width,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,C,Area,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10127163,C,Separation,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10232160,L,Concentration,0,0,1,1,1,1,LC,...,1,1,2,1,1,1,1,1,1,4
10232160,L,Width,0,0,0,0,1,1,LC,...,1,1,0,1,1,1,1,1,1,4


[1] "sid"                       "sim"                      
 [3] "variable"                  "pre"                      
 [5] "main"                      "cvs_graph"                
 [7] "cvs_table"                 "qual_score"               
 [9] "quant_score"               "activity_order"           
[11] "sim_index"                 "similar_sim"              
[13] "lab_experience"            "prior_number_virtual_labs"
[15] "overall.POCC"              "PC1"                      
[17] "PC2"                       "pre_with_ident"           
[19] "main_with_ident"           "CVS_context"              
[21] "use_table"                 "use_graph"                
[23] "use_concentration"         "use_width"                
[25] "use_area"                  "use_separation"           
[27] "use_all_vars"

# Stat model 1: Predicting main model scores

We try to predict the type of model in the main worksheet (0,1 or 2 for neither, qual or quant). To do so, we have 10 independant variables:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)
* pre (0,1,2)

We could add student attributes but since these are mostly unbalanced, we ignore (lab_experience, similar_sim, prior_number_virtual_labs, attitude components)

Some resources:
* On SS Types: https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/
* on drop() function to do type 3: https://www.statmethods.net/stats/anova.html
* On repeated measures: http://psych.wisc.edu/moore/Rpdf/610-R8_OneWayWithin.pdf, https://datascienceplus.com/two-way-anova-with-repeated-measures/
* the car package: https://cran.r-project.org/web/packages/car/car.pdf

In [4]:
lm1 = lm(main
        ~  cvs_table + cvs_graph + variable + sim_index + pre + sid,
         data=mydata)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

Warning message:
"package 'bindrcpp' was built under R version 3.4.4"

term,sumsq,df,statistic,p.value,eta
cvs_table,2.875905e-03,1,0.01265297,9.104905e-01,2.915345e-05
cvs_graph,2.846821e+00,1,12.52501086,4.449018e-04,2.804996e-02
variable,2.341790e+00,3,3.43434961,1.698431e-02,2.318924e-02
sim_index,2.769224e+00,1,12.18360845,5.314560e-04,2.730627e-02
pre,5.063759e-01,1,2.22787537,1.362667e-01,5.107137e-03
sid,1.003316e+02,146,3.02345002,6.702068e-19,5.042401e-01
Residuals,9.864427e+01,434,NA,NA,5.000000e-01


We see that, in order of significance: cvs_graph, sim_index, and variable matter.

# Stat model 2: Predicting transfer data

In [5]:
lm1 = lm(quant_score
        ~  cvs_table + cvs_graph + variable + sim_index + pre + sid,
         data=mydata)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,3.560571e-04,1,0.002591862,9.594204e-01,5.971997e-06
cvs_graph,3.313290e-01,1,2.411858311,1.211482e-01,5.526565e-03
variable,1.514087e+00,3,3.673853147,1.229926e-02,2.476635e-02
sim_index,2.417089e-01,1,1.759482309,1.853867e-01,4.037737e-03
pre,4.862882e-01,1,3.539859443,6.057962e-02,8.090370e-03
sid,6.299217e+01,146,3.140696461,3.782563e-20,5.137482e-01
Residuals,5.962075e+01,434,NA,NA,5.000000e-01


As expected, CVS doesn't predict quant transfer scores, only variable does.

# Stat model 3: Predicting the use of CVS

## For cvs_table

In [6]:
# mydata$variable <- relevel(mydata$variable, "Width")
mixed <- glmer(
    cvs_table
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_table ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + overall.POCC + PC1 + PC2 + (1 |      sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   636.9    693.8   -305.4    610.9      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7511 -0.4307  0.2088  0.3572  2.3885 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 5.991    2.448   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)               -1.20670    1.87139  -0.645  0.51905   
variableConcentration      0.53139    0.36369   1.461  0.14399   
variableSeparation        -0.25492    0.33945  -0.751  0.45267   
variableWidth              0.2

## For cvs_graph

In [7]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_graph ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + overall.POCC + PC1 + PC2 + (1 |      sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   605.7    662.6   -289.9    579.7      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0994 -0.3173 -0.1006  0.3446  2.8043 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 10.38    3.222   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -6.0676     2.3957  -2.533  0.01132 *  
variableConcentration       0.7346     0.4053   1.812  0.06994 .  
variableSeparation         -0.1027     0.3733  -0.275  0.78319    
variableWidth             